In [13]:
from openai import OpenAI
import gradio as gr

import os
from dotenv import load_dotenv
from IPython.display import display, Markdown

In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
openai = OpenAI(api_key = openai_api_key)

In [5]:
def print_markdown(text):
    """Better output format for notebooks"""
    display(Markdown(text))

In [ ]:
def get_ai_tutor_response(user_question):
    """
    This function sends  question to OpenAI, asking it to respond as an AI Tutor

    Args:
        user_question (str): The question asked  by the user

    Returns:
        str: The OpenAI response, or an error message
    """
    # Instruct OpenAI on how to respond
    system_prompt = "You are a helpful and patient AI Tutor.  Explain concepts clearly and consisely."

    try:
        # Initiate API call
        response = openai.chat.completions.create(
            model = 'gpt-4o-mini',
            messages = [{"role":"system", "content": system_prompt}, {"role": "user", "content": user_question}],
            temperature = 0.7, # some creativity, but keep responses focused
        )
        # Get the answer content
        ai_response = response.choices[0].message.content
        return ai_response
    
    except Exception as e:
        # Error handling
        print(f"An error occurred: {e}")
        return(f"Sorry, I encountered an error trying to get an answer: {e}")

In [ ]:
# User question logic
test_question = "Could you explain the concept of classes in Python and their purpose in programming?"
print(f"Asking the AI Tutor: '{test_question}")

# Call the function that submits the question to OpenAI with the user question
tutor_answer = get_ai_tutor_response(test_question)

# Output the OpenAI response
print_markdown("\nAI Tutor's Response:\n")
print_markdown(tutor_answer)

Asking the AI Tutor: 'Could you explain the concept of classes in Python and their purpose in programming?



Ai Tutor's Response:


Certainly! In Python, a **class** is a blueprint for creating objects. An object is an instance of a class. Classes allow you to bundle data (attributes) and functionality (methods) together. This is a key part of **object-oriented programming (OOP)**, which is a programming paradigm that uses "objects" to design applications and programs.

### Purpose of Classes:

1. **Encapsulation**: Classes help to encapsulate data and functions that operate on that data. This means that you can group related variables (attributes) and functions (methods) together, which makes your code more organized and easier to manage.

2. **Reusability**: Once a class is defined, you can create multiple objects (instances) from it. This promotes code reuse, as you don't have to write the same code multiple times. You can create various instances of a class that share the same structure and behavior but hold different data.

3. **Inheritance**: Classes support inheritance, allowing you to create a new class based on an existing class. This helps to build a hierarchy and enables code reuse and extension of existing functionality.

4. **Polymorphism**: With classes, you can define methods in a way that they can behave differently based on the object calling them. This allows for flexible and interchangeable code.

### Defining a Class:

Here's a basic example of how to define a class in Python:

```python
class Dog:
    # Constructor to initialize attributes
    def __init__(self, name, age):
        self.name = name  # Attribute
        self.age = age    # Attribute

    # Method to make the dog bark
    def bark(self):
        return f"{self.name} says Woof!"

# Creating an instance (object) of the Dog class
my_dog = Dog("Buddy", 3)

# Accessing attributes and methods
print(my_dog.name)  # Output: Buddy
print(my_dog.bark())  # Output: Buddy says Woof!
```

### Key Components of a Class:

- **Attributes**: Variables that hold data related to the object (e.g., `name`, `age`).
- **Methods**: Functions defined within a class that operate on its attributes (e.g., `bark()`).
- **Constructor (`__init__`)**: A special method that is called when an object is created. It initializes the object's attributes.

### Conclusion:

Classes in Python are fundamental for organizing code, promoting reuse, and enabling powerful design patterns. They help you model real-world entities and their interactions in a structured way, making your programs more maintainable and scalable.

# Build Gradio interface

In [22]:
"""
fn: get_ai_turor_response function
inputs: Component for user input / question
outputs: Component to display the AI's answer
title / description: Text for the UI heading
"""
ai_tutor_interface_simple = gr.Interface(
    fn = get_ai_tutor_response,
    inputs = gr.Textbox(lines=5, placeholder="Ask the AI Tutor anything!", label="Your Question"),
    outputs = gr.Textbox(label="AI Tutor's Answer"),
    title = "AI Tutor",
    description = "Enter your qustion below and the AI Tutor will provide an explaination power by OpenAI!",
    flagging_mode = "never"
)

print("Launching AI Tutor interface....")
ai_tutor_interface_simple.launch()

Launching AI Tutor interface....
* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


# Add streaming output

In [24]:
def stream_ai_tutor_response(user_question):
    """
    Sends a question to the OpenAI API and streams a response as a generator

    Args:
        user_question (str): The question asked  by the user

    Yields:
        str: Chunks of the AI's response 
    """

    system_prompt = "You are a helpful and patient AI Tutor.  Explain concepts clearly and consisely."

    try:
        stream = openai.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_question}],
            temperature = 0.7,
            stream = True,
        )

        # Empty list we will use to store the full response
        full_response = ""

        # Iterate through each response chunk as its recieved
        for chunk in stream:
            # Verify contents
            if chunk.choices[0].delta and chunk.choices[0].delta.content:
                # extract the text from current chunk
                text_chunk = chunk.choices[0].delta.content
                # Append this chunk to the overall response
                full_response += text_chunk
                # Send current state of the response to Gradio UI as it arrives
                yield full_response

    except Exception as e:
        print(f"An error occurred during streaming: {e}")
        yield f"Encoutnered an error: {e}"

# Updated Gradio response for use with streaming functionality

In [ ]:
"""
fn: stream_ai_tutor_response function
inputs: Component for user input / question
outputs: Component to display the AI's answer
title / description: Text for the UI heading
"""
ai_tutor_interface_streaming = gr.Interface(
    fn = stream_ai_tutor_response,
    inputs = gr.Textbox(lines=5, placeholder="Ask the AI Tutor anything!", label="Your Question"),
    outputs = gr.Textbox(label="AI Tutor's Answer"),
    title = "AI Tutor",
    description = "Enter your qustion below and the AI Tutor will provide an explaination power by OpenAI!",
    flagging_mode = "never"
)

print("Launching AI Tutor interface....")
ai_tutor_interface_streaming.launch()

Launching AI Tutor interface....
* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


# Add explaination level slider bar

In [28]:
# define a map of explaination levels
explanation_levels = {
    1: "very simple, as if I am a 5 year old",
    2: "simple, as if I am 10 years old",
    3: "high school level equivalent",
    4: "college student level equivalent",
    5: "expert or PHD level in this field"
}

In [ ]:
def stream_ai_tutor_with_level(user_question, explanation_level_value):
    """
    Sends a question to the OpenAI API and streams a response as a generator

    Args:
        user_question (str): The question asked  by the user
        explanation_level_value (int): The value from the slider bar (1-5)

    Yields:
        str: Chunks of the AI's response 
    """

    # Get the descriptive text from the chosen level, default to clearly and concisely
    level_description = explanation_levels.get(
        explanation_level_value = "clearly and concisely" 
    )

    # Construct the system prompt based on the user question and the explanation level
    system_prompt = f"You are a helpful AI Totor.  Explain the following concept {level_description}"

    print(f"DEUBUG: Using System Prompt: '{system_prompt}'")

    try:
        stream = openai.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_question}],
            temperature = 0.7,
            stream = True,
        )

        # Empty list we will use to store the full response
        full_response = ""

        # Iterate through each response chunk as its recieved
        for chunk in stream:
            # Verify contents
            if chunk.choices[0].delta and chunk.choices[0].delta.content:
                # extract the text from current chunk
                text_chunk = chunk.choices[0].delta.content
                # Append this chunk to the overall response
                full_response += text_chunk
                # Send current state of the response to Gradio UI as it arrives
                yield full_response

    except Exception as e:
        print(f"An error occurred during streaming: {e}")
        yield f"Encoutnered an error: {e}"

In [41]:
# New Gradiop UI with Textbox and Slider Bar
ai_tutor_interface_slider = gr.Interface(
    fn = stream_ai_tutor_with_level,
    inputs = [
        gr.Textbox(lines=5, placeholder="Ask the AI Tutor a question...", label="Your Question"),
        gr.Slider(
            minimum=1,
            maximum=5,
            step=1, # allow only whole number increments
            value=3,
            label="Explanation Level"
        ),
    ],
    outputs = gr.Markdown(label="AI Tutor's Answer", container=True, height=250),
    title = "Advanced AI Tutor",
    description = "Ask a question, and select the desired level of explaination using the slider bar",
    flagging_mode = "never"
)

print("Lanching Addvaced AI Tutor Interface...")
ai_tutor_interface_slider.launch()

Lanching Addvaced AI Tutor Interface...
* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


DEUBUG: Using System Prompt: 'You are a helpful AI Totor.  Explain the following concept college student level equivalent'
DEUBUG: Using System Prompt: 'You are a helpful AI Totor.  Explain the following concept very simple, as if I am a 5 year old'
DEUBUG: Using System Prompt: 'You are a helpful AI Totor.  Explain the following concept expert or PHD level in this field'
